# Building web UIs with Flask

In this module, we'll see how to build a basic web UI using Flask

First, let's get our helper functions in place to be able to run Flask in the notebook:

In [1]:
import os, sys, time, threading, subprocess, contextlib

def output_thread(proc):
    for line in proc.stdout:
        print(line.decode('utf-8'), end='')
    print('Exiting output thread')

def run_flask_app(app_name):
    proc = subprocess.Popen(
        # [sys.executable, 'flask', 'run'],
        ['flask', 'run', '--no-reload'],
        env={
            **os.environ, 
            'FLASK_APP': app_name,
            'FLASK_ENV': 'development',
        },
        stderr=subprocess.STDOUT,
        stdout=subprocess.PIPE
    )
    # Wait for the port to bind
    for line in proc.stdout:
        line = line.decode('utf-8')
        print(line, end='')
        if ' * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)' in line:
            break
    else:
        print('== Error starting server ==')
        return None
    thd = threading.Thread(target=output_thread, args=(proc,))
    thd.setDaemon(True)
    thd.start()
    return proc


@contextlib.contextmanager
def running_app(app_name):
    proc = run_flask_app(app_name)
    try:
        yield proc
    finally:
        proc.kill()        

# Flask and Jinja

Although you can use any templating library you like with Flask, it comes with good support for Jinja2

In [2]:
%%file data/flask-examples/ui1.py

from flask import Flask, render_template

app = Flask(__name__)

@app.route('/')
@app.route('/<name>')
def hello(name=None):
    return render_template('hello.html', name=name)

Overwriting data/flask-examples/ui1.py


In [3]:
%%file data/flask-examples/templates/hello.html
<!doctype html>
<title>Hello Flask</title>
{% if name %}
<h1>Hello {{ name }}!</h1>
{% else %}
<h1>Hello, World!</h1>
{% endif %}

Overwriting data/flask-examples/templates/hello.html


In [6]:
import webbrowser

In [7]:
with running_app('data.flask-examples.ui1'):
    webbrowser.open('http://localhost:5000')
    webbrowser.open('http://localhost:5000/General Kenobi')    
    time.sleep(2)

 * Serving Flask app "data.flask-examples.ui1"
 * Environment: development
 * Debug mode: on
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Jul/2020 13:06:28] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2020 13:06:28] "GET /favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2020 13:06:29] "GET /General%20Kenobi HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2020 13:06:29] "GET /favicon.ico HTTP/1.1" 200 -
Exiting output thread


### Form data

We can display read-only output pretty easily, but it's nicer to allow form submission:


In [8]:
%%file data/flask-examples/ui2.py

from flask import Flask, render_template, request

app = Flask(__name__)

@app.route('/')
def home():
    return render_template('home.html')


@app.route('/form')
def form():
    return render_template('form.html')
    

@app.route('/post', methods=['POST'])
def post():
    return render_template('post.html', form=request.form)

Overwriting data/flask-examples/ui2.py


In [9]:
%%file data/flask-examples/templates/home.html
<!doctype html>
<title>Hello Flask</title>

<h1>You can visit the following links:</h1>
<ul>
    <li><a href="/form">The Form</a></li>
</ul>

Overwriting data/flask-examples/templates/home.html


In [10]:
%%file data/flask-examples/templates/form.html
<!doctype html>
<title>Hello Form!</title>

Return <a href="/">home</a>

<form method="POST" action="/post">
  <label for="name">Your name</label><br/>
  <input id="name" name="name"/><br/>

  <label for="color">Your favorite color</label><br/>
  <input id="color" name="color"/><br/>

  <input type="submit"/>
</form>

Overwriting data/flask-examples/templates/form.html


In [11]:
%%file data/flask-examples/templates/post.html
<!doctype html>
<title>Hello Post!</title>

Return <a href="/">home</a>

You entered the following:

<div style="background-color:{{form.color}}">
Name: {{form.name}}<br/>
Favorite Color: {{form.color}}
</div>

Overwriting data/flask-examples/templates/post.html


In [12]:
sp = run_flask_app('data.flask-examples.ui2')
webbrowser.open('localhost:5000')

 * Serving Flask app "data.flask-examples.ui2"
 * Environment: development
 * Debug mode: on
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


True

127.0.0.1 - - [29/Jul/2020 13:10:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2020 13:10:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2020 13:10:34] "GET /form HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2020 13:10:34] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [29/Jul/2020 13:10:41] "POST /post HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2020 13:10:41] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [29/Jul/2020 13:11:20] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2020 13:11:21] "GET /form HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2020 13:11:32] "POST /post HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2020 13:11:40] "GET / HTTP/1.1" 200 -


In [13]:
sp.kill()

Exiting output thread


# Adding some state

Now that we have html and forms, we can build a little app:

In [14]:
%%file data/flask-examples/state.py
import os, threading, pickle


class StateManager():
    
    def __init__(self, filename):
        self.filename = filename
        self.lock = threading.Lock()
        self._state = None
        
    def get(self):
        with self.lock:
            if self._state is None:
                self._state = self._load_state()
        return self._state

    def save(self):
        with self.lock:
            self._persist_state()
            
    def _load_state(self):
        if not os.path.exists(self.filename):
            return {}
        with open(self.filename, 'rb') as f:
            return pickle.load(f)
        
    def _persist_state(self):
        with open(self.filename, 'wb') as f:
            pickle.dump(self._state, f)
            

Overwriting data/flask-examples/state.py


In [ ]:
%%file data/flask-examples/todo.py
import os
from flask import Flask, render_template, request, url_for, redirect

from .state import StateManager

app = Flask(__name__)

STATE_MANAGER = StateManager(
    os.path.abspath(
        os.path.join(
            os.path.dirname('__file__'),
            'state.pkl'
        )
    )
)

@app.route('/')
def home():
    state = STATE_MANAGER.get()
    items = state.get('items', [])
    return render_template('todo.html', items=items)


@app.route('/add', methods=['POST'])
def add_todo():
    state = STATE_MANAGER.get()
    items = state.setdefault('items', [])
    items.append(request.form['item'])
    STATE_MANAGER.save()
    return redirect(url_for('home'))


@app.route('/del', methods=['POST'])
def del_todo():
    print(f'Delete {request.form["index"]}')
    state = STATE_MANAGER.get()
    index = int(request.form['index'])
    items = state.get('items', [])
    if index < len(items):
        del items[index]
        STATE_MANAGER.save()
    return redirect(url_for('home'))


In [15]:
%%file data/flask-examples/templates/todo.html
<!doctype html>
<title>Simple Todo</title>

<h1>To-Do List</h1>

<ul>
{% for item in items %}
    <li>
        {{item}}
        <form method="POST" action={{url_for('del_todo')}}>
            <input type="hidden" name="index" value="{{loop.index0}}"/>
            <input type="submit" value="Delete"/>
        </form>
    </li>
{% endfor%}
</ul>

<form method="POST" action={{url_for('add_todo')}}>
    <label for="item">Add todo item</label><br/>
    <input name="item"/>
</form>


Overwriting data/flask-examples/templates/todo.html


In [16]:
sp = run_flask_app('data.flask-examples.todo')

 * Serving Flask app "data.flask-examples.todo"
 * Environment: development
 * Debug mode: on
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Jul/2020 13:19:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2020 13:19:16] "POST /del HTTP/1.1" 302 -
127.0.0.1 - - [29/Jul/2020 13:19:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2020 13:19:17] "POST /del HTTP/1.1" 302 -
127.0.0.1 - - [29/Jul/2020 13:19:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2020 13:19:18] "POST /del HTTP/1.1" 302 -
127.0.0.1 - - [29/Jul/2020 13:19:18] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2020 13:19:28] "POST /add HTTP/1.1" 302 -
127.0.0.1 - - [29/Jul/2020 13:19:28] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2020 13:19:40] "POST /add HTTP/1.1" 302 -
127.0.0.1 - - [29/Jul/2020 13:19:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2020 13:19:44] "POST /add HTTP/1.1" 302 -
127.0.0.1 - - [29/Jul/2020 13:19:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2020 13:19:48] "POST

In [17]:
sp.kill()

Exiting output thread


# Lab

Open the [Flask UI lab][flask-ui-lab]

[flask-ui-lab]: ./flask-ui-lab.ipynb